In [1]:
import pandas as pd
from nltk import sent_tokenize
# encoding 문제시, engine = 'python' 입력할 것
# data = pd.read_excel('outputVer2.xlsx')
data = pd.read_csv('camera_reviews.csv', engine = 'python')
data.head()

,product_name,asin,review,review_date,score,user
0,pdc-640,B00000J48G,"I have really enjoyed using this camera, for a...","07 15, 2000",4.0,"""111jbd"""
1,pdc-640,B00000J48G,I've had my camera for two months and I love i...,"11 18, 1999",4.0,A.J.Kirchoff (xgi@ametro.net)
2,pdc-640,B00000J48G,I bought this camera a few months ago and love...,"06 10, 2002",4.0,"Andrea Rowland ""Andrea"""
3,pdc-640,B00000J48G,I bought this as a first time digital camera p...,"04 26, 2000",4.0,"""denisey65"""
4,pdc-640,B00000J48G,This is great value for the price. I was look...,"10 11, 2000",5.0,"Dennis Frank ""djarchivist"""


In [2]:
import pandas as pd
from nltk import sent_tokenize
import re

def make_same_feature(text):
    text=text.lower()
    # 각각의 feature 별 동의어를 대표명으로 정리
   
    text=re.sub('resolution|pixel|megapixel', 'pixel', str(text))
    text=re.sub('lens|wide\sangle|normal\srange', 'lens', str(text))
    text=re.sub('optical|zoom|optical\szoom|digital\szoom', 'optical', str(text))
    text=re.sub('memory|megabytes|mb', 'memory', str(text))
    text=re.sub('burst|continuous|shutter|recovery|motion|sport', 'burst', str(text))
    text=re.sub('battery|batteries|power', 'battery', str(text))
    text=re.sub('focus|exposure|manual|iso', 'focus', str(text))
    text=re.sub('lcd|screen', 'lcd', str(text))
    text=re.sub('compression|compress|jpeg', 'compression', str(text))
    text=re.sub('flash|light', 'flash', str(text))
    return text

def pre_processing(data):
    # score 변수를 정리하여 현재 점수만을 표현하도록 변경
#     data['score'] = data['score'].apply(lambda x : x.split(' ')[0])
    
    result = []
    for i in range(len(data)):
        # 아래 for문에서 딕셔너리를 만드릭 위해 각각의 데이터를 변수로 저장
        asin = data.loc[i, 'asin']
#         model = data.loc[i, 'item_model_no']
        product = data.loc[i, 'product_name']
        date = data.loc[i, 'review_date']
        score = data.loc[i, 'score']
        user = data.loc[i, 'user']
        text = data.loc[i, 'review']
        

        Sentences_list = sent_tokenize(text)

        for txt in Sentences_list:
            dict_n = {
                'product' : product,
                'asin' : asin,
#                 'model' : model,
                'date' : date,
                'score' : score,
                'user' : user,
                'review' : txt
            }
            result.append(dict_n)
    print(len(result))
    # 데이터프레임으로 형식 전환
    df = pd.DataFrame(result)
    
    # 각 feature별 동의어 처리 함수 사용
    df['review'] = df['review'].apply(make_same_feature)
    
    # feature가 들어있는 문장만 별로로 출력
    # str은 앞의 시리즈 데이터를 문자열로 처리하기 위한 변환식
    # contains는 데이터 내에 패턴이 존재하기만 하면 위치에 상관없이 True 값 출력
    df = df[df.review.str.contains('pixel|lens|optical|memory|burst|battery|focus|lcd|compression|flash') == True]
    df = df.reset_index(drop = True)
    return df

In [3]:
data_pp = pre_processing(data)
data_pp.head()

1121028


,asin,date,product,review,score,user
0,B00000J48G,"11 18, 1999",pdc-640,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net)
1,B00000J48G,"11 18, 1999",pdc-640,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net)
2,B00000J48G,"06 10, 2002",pdc-640,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea"""
3,B00000J48G,"04 26, 2000",pdc-640,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65"""
4,B00000J48G,"04 26, 2000",pdc-640,"flashing is important in any photography, and...",4.0,"""denisey65"""


In [4]:
data_pp.to_csv('reviews_preprocessing.csv', encoding='utf-8', index=False)

In [8]:
data_pp

,asin,date,product,review,score,user
0,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i have taken over 500 pics using all three foc...,4.0,A.J.Kirchoff (xgi@ametro.net)
1,B00000J48G,"11 18, 1999",Polaroid PDC-640 0.3MP Digital Camera Creative...,i wish that it would take the 16memory and th...,4.0,A.J.Kirchoff (xgi@ametro.net)
2,B00000J48G,"06 10, 2002",Polaroid PDC-640 0.3MP Digital Camera Creative...,only draw back is theres no optical lens.,4.0,"Andrea Rowland ""Andrea"""
3,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,i like the way my 5x7 prints and smaller turn...,4.0,"""denisey65"""
4,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,"flashing is important in any photography, and...",4.0,"""denisey65"""
5,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,you definitely cannot rush the recharge on th...,4.0,"""denisey65"""
6,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,also- the &quot;video&quot; function on the l...,4.0,"""denisey65"""
7,B00000J48G,"04 26, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,i bought an optional flashpath adapter that a...,4.0,"""denisey65"""
8,B00000J48G,"10 11, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,a flashpath floppy disk adapter makes it easy ...,5.0,"Dennis Frank ""djarchivist"""
9,B00000J48G,"02 21, 2000",Polaroid PDC-640 0.3MP Digital Camera Creative...,"anyway, this camera quite good, but i thought...",4.0,d
